# Some important data transformation tools

## Multi Index, Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
np.random.seed(1)
# manual multi-index creation:
# provide the index argument with a list of lists
data = pd.Series(np.random.randint(10, size = 9), 
                 index = [['a','a','a','b','b','b','c','c','c'],
                          [ 1 , 2 , 3 , 1 , 2 , 3 , 1 , 2 , 3 ]])

In [6]:
data

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32

In [7]:
data.index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [8]:
# select via the outer index
data.loc['b']

1    5
2    0
3    0
dtype: int32

In [11]:
# select via the inner index
data.loc[:,2] 

a    8
b    0
c    7
dtype: int32

In [12]:
# the unstack function returns a new DataFrame where the values have been unstacked
# similar to tidyr's spread() function in R
data.unstack()

,1,2,3
a,5,8,9
b,5,0,0
c,1,7,6


In [13]:
# after unstacking, the index is no longer a multi index
data.unstack().index

Index(['a', 'b', 'c'], dtype='object')

In [14]:
data.unstack().shape

(3, 3)

In [15]:
# the inverse operation of unstack() is stack()
# applying both of these functions will return the same series
data.unstack().stack()

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32

In [16]:
# you can swap the levels of the multi index using swaplevel
data.swaplevel()

1  a    5
2  a    8
3  a    9
1  b    5
2  b    0
3  b    0
1  c    1
2  c    7
3  c    6
dtype: int32

In [17]:
# the .loc accessors work as expected
data.swaplevel().loc[:,'a']

1    5
2    8
3    9
dtype: int32

In [18]:
# swaplevel will keep the original order
# you may want to sort based on the new swapped index levels
# you must save the output as data remains unchanged
data.swaplevel().sort_index()

1  a    5
   b    5
   c    1
2  a    8
   b    0
   c    7
3  a    9
   b    0
   c    6
dtype: int32

In [19]:
print(data)

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32


In [20]:
# summing and other aggregate functions can be performed on an index-based level
# calling sum() on a series, will sum the whole series
data.sum()

41

In [21]:
# you can call sum on the level 0 (the first level of the index)
# we get sums for each value in the first level of the index
data.sum(level = 0)

a    22
b     5
c    14
dtype: int32

In [ ]:
data.sum(level = 1)

# Reshaping and Pivoting Data

In [22]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index  = pd.Index(['alpha', 'beta'], name='letter'),
                    columns= pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


In [23]:
data.stack()  # creates a multi-index

letter  number
alpha   one       0
        two       1
        three     2
beta    one       3
        two       4
        three     5
dtype: int32

In [24]:
data.stack().unstack()  # unstack undoes the creation of the stacks

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


In [25]:
data.stack().unstack(0) # you can specify how the unstacking should be done
# here we specify that we should unstack the first level of the multi-index

letter,alpha,beta
number,,
one,0,3
two,1,4
three,2,5


In [26]:
data.stack().unstack('letter')
# you can specify the unstacking by the index level name

letter,alpha,beta
number,,
one,0,3
two,1,4
three,2,5


In [27]:
data.stack().unstack('number')

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


### Unstacking can introduce missing values

In [31]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])  
# using the argument keys when concat series will produce a multi-index
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [32]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [33]:
data2.unstack().stack() # stack() will filter out missing values

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [34]:
data2.unstack().stack(dropna = False) # you can force stack to keep the NaNs

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

# Small example data wrangling

In [36]:
data = pd.read_csv('macrodata.csv')

https://www.statsmodels.org/dev/datasets/generated/macrodata.html

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 14 columns):
year        203 non-null float64
quarter     203 non-null float64
realgdp     203 non-null float64
realcons    203 non-null float64
realinv     203 non-null float64
realgovt    203 non-null float64
realdpi     203 non-null float64
cpi         203 non-null float64
m1          203 non-null float64
tbilrate    203 non-null float64
unemp       203 non-null float64
pop         203 non-null float64
infl        203 non-null float64
realint     203 non-null float64
dtypes: float64(14)
memory usage: 22.3 KB


In [38]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.PeriodIndex.html

In [40]:
# We can create a time based index of periods consisting of the year and quarter
periods = pd.PeriodIndex(year = data.year, quarter = data.quarter, name = 'date')

In [41]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [42]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [44]:
data = data.reindex(columns = columns)

In [45]:
periods.to_timestamp('D','end')  # changes 1959Q1 to a date: the end date of Q1 of 1959: 1959-03-31

DatetimeIndex(['1959-03-31 23:59:59.999999999',
               '1959-06-30 23:59:59.999999999',
               '1959-09-30 23:59:59.999999999',
               '1959-12-31 23:59:59.999999999',
               '1960-03-31 23:59:59.999999999',
               '1960-06-30 23:59:59.999999999',
               '1960-09-30 23:59:59.999999999',
               '1960-12-31 23:59:59.999999999',
               '1961-03-31 23:59:59.999999999',
               '1961-06-30 23:59:59.999999999',
               ...
               '2007-06-30 23:59:59.999999999',
               '2007-09-30 23:59:59.999999999',
               '2007-12-31 23:59:59.999999999',
               '2008-03-31 23:59:59.999999999',
               '2008-06-30 23:59:59.999999999',
               '2008-09-30 23:59:59.999999999',
               '2008-12-31 23:59:59.999999999',
               '2009-03-31 23:59:59.999999999',
               '2009-06-30 23:59:59.999999999',
               '2009-09-30 23:59:59.999999999'],
              dtype=

In [46]:
data.index = periods.to_timestamp('D','end')

In [50]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [51]:
data.stack().head(10)  # stack creates a series

date                           item   
1959-03-31 23:59:59.999999999  realgdp    2710.349
                               infl          0.000
                               unemp         5.800
1959-06-30 23:59:59.999999999  realgdp    2778.801
                               infl          2.340
                               unemp         5.100
1959-09-30 23:59:59.999999999  realgdp    2775.488
                               infl          2.740
                               unemp         5.300
1959-12-31 23:59:59.999999999  realgdp    2785.204
dtype: float64

In [52]:
data.stack().reset_index().head()
# calling reset index turns the current index into a new column and creates a new index

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


In [57]:
ldata = data.stack().reset_index().rename(columns = {0: 'value'})

# rename changes the column title '0' to 'value'
ldata.head(10)

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [ ]:
# unstack doesn't work, because the stacking and unstacking is powered by multi-index
ldata.unstack()

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [ ]:
# if the data is in 'long' form, you can change it to 'wide' form with pivot
ldata.pivot('date','item','value').head()

In [ ]:
data.head()

# Group By


In [ ]:
np.random.seed(1)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(20, size = 5),
                   'data2' : np.random.randint(20, size = 5)})
df

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [ ]:
grouped.mean()

In [ ]:
df.groupby(df['key1']).mean()  
# if you don't specify the column, it'll apply the function to the entire dataframe

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
# means has a multi-index

In [ ]:
# with the multi-index, you can unstack
means.unstack()

In [ ]:
# you can perform group by on Series that are not in the dataframe, but are of the correct length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

In [ ]:
# groupby applied to the entire dataframe, not just one column
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

In [ ]:
df.groupby(['key1', 'key2']).size()

### Iterating over groups

In [ ]:
df

In [ ]:
# the groupby creates a series of tuples that can be unpacked into name and group
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print('----------------')

In [ ]:
for name, group in df.groupby('key2'):
    print(name)
    print(group)
    print(group.sum())
    print('----------------')
